# Assembly

Humberto Ortiz Zuazaga

Example code for chapter 3.

# ba3a

Compute the kmer composition of text


In [1]:
def Composition(k, text):
    kmers = []
    for i in range(len(text) - k + 1):
        kmers.append(text[i:i+k])
    return sorted(kmers)

In [2]:
# prueba del libro
Composition(3, "TATGGGGTGC")

['ATG', 'GGG', 'GGG', 'GGT', 'GTG', 'TAT', 'TGC', 'TGG']

# Representar grafos

Vamos a usar objetos de python para representar grafos usando listas de adyacencia.

Input: una lista de k-mers.   
Output: el grafo de solape (overlap) (con flechitas)

In [3]:
class OverlapGraph():
    # para crear un OverlapGraph
    def __init__(self, pattern):
        self.pattern = pattern
        self.index = 0
        self.G = {}
        k = len(pattern[0])

        for primero in pattern:
            if primero not in self.G:
                self.G[primero] = set()
            for segundo in pattern:
                # check if end of primero is same as begining of segundo
                if primero[-(k-1):] == segundo[:k-1]:
                    self.G[primero].add(segundo)
    def print(self):
        "Para imprimir un overlap graph como lo quiere rosalind.info"
        for kmer, edges in sorted(self):
            for edge in edges:
                print(kmer, "->", edge)

    # metodos especiales de python para iteracion (for loop)
    def __iter__(self):
        return self
    
    # pasar al proximo elemento
    def __next__(self):
        if self.index == len(self.pattern):
            self.index = 0
            raise StopIteration
            
        kmer = self.pattern[self.index]
        edges = list(self.G[kmer])
        self.index = self.index + 1
        return (kmer, edges)
    
    # extraer la lista de adyacencia de un elemento
    def __getitem__(self, kmer):
        return list(self.G[kmer])

In [4]:
G = OverlapGraph(Composition(3, "TAATGCCATGGGATGTT"))

In [5]:
list(G)

[('AAT', ['ATG']),
 ('ATG', ['TGC', 'TGT', 'TGG']),
 ('ATG', ['TGC', 'TGT', 'TGG']),
 ('ATG', ['TGC', 'TGT', 'TGG']),
 ('CAT', ['ATG']),
 ('CCA', ['CAT']),
 ('GAT', ['ATG']),
 ('GCC', ['CCA']),
 ('GGA', ['GAT']),
 ('GGG', ['GGA', 'GGG']),
 ('GTT', []),
 ('TAA', ['AAT']),
 ('TGC', ['GCC']),
 ('TGG', ['GGA', 'GGG']),
 ('TGT', ['GTT'])]

In [6]:
G.print()

AAT -> ATG
ATG -> TGC
ATG -> TGT
ATG -> TGG
ATG -> TGC
ATG -> TGT
ATG -> TGG
ATG -> TGC
ATG -> TGT
ATG -> TGG
CAT -> ATG
CCA -> CAT
GAT -> ATG
GCC -> CCA
GGA -> GAT
GGG -> GGA
GGG -> GGG
TAA -> AAT
TGC -> GCC
TGG -> GGA
TGG -> GGG
TGT -> GTT


In [7]:
pattern = Composition(3, "TAATGCCATGGGATGTT")
OverlapGraph(pattern).print()

AAT -> ATG
ATG -> TGC
ATG -> TGT
ATG -> TGG
ATG -> TGC
ATG -> TGT
ATG -> TGG
ATG -> TGC
ATG -> TGT
ATG -> TGG
CAT -> ATG
CCA -> CAT
GAT -> ATG
GCC -> CCA
GGA -> GAT
GGG -> GGA
GGG -> GGG
TAA -> AAT
TGC -> GCC
TGG -> GGA
TGG -> GGG
TGT -> GTT


# De Bruijn graph

Try solving problem BA3D from rosalind.info.

In [8]:
def debruijn(filename):
    with open(filename) as f:
        k = int(f.readline())
        #print(k)
        secuencia = f.readline().strip()
        G = {}
        for i in range(len(secuencia)-k+1):
            kmero = secuencia[i:i+k]
            principio = kmero[:-1]
            final = kmero[1:]
            if principio not in G:
                G[principio] = [final]
            else:
                G[principio].append(final)
    # el grafo esta en G
    # imprimir bonito al archivo "results.txt"
    
    with open("results.txt", mode="w") as f:
        keys = G.keys()
    
        for kmer in sorted(keys):
            edges = G[kmer]
            print(kmer, "->", ",".join(sorted(edges)), file=f)

In [9]:
# cambia el nombre del archivo
debruijn("foo.txt")

# Pruebas

Probando como seleccionar partes de un kmero.

In [10]:
kmero = "AAGA"

In [11]:
kmero[:-1]

'AAG'

In [12]:
kmero[1:]

'AGA'

In [13]:
edges = ['CTC', 'CTA']

In [14]:
edges

['CTC', 'CTA']

In [15]:
",".join(edges)

'CTC,CTA'

In [16]:
# Ejemplo de rosalind.info
pattern = ["ATGCG",
"GCATG",
"CATGC",
"AGGCA",
"GGCAT"]

In [17]:
OverlapGraph(pattern).print()

AGGCA -> GGCAT
CATGC -> ATGCG
GCATG -> CATGC
GGCAT -> GCATG


In [18]:
a = [5, 4, 1]

In [19]:
a[-2:]

[4, 1]

In [20]:
a[0:1]

[5]

In [21]:
[4, 1] + [5]

[4, 1, 5]

# Eulerian Cycle

Take a stab at BA3F from rosalind.info.

Read a file like:

```
0 -> 3
1 -> 0
2 -> 1,3
3 -> 2
```

In [22]:
def read_ba3f(filename):
    G = {}
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            fields = line.strip().split(" -> ")
            head = fields[0]
            edges = fields[1].split(",")
            G[head] = edges
    return G

In [23]:
read_ba3f("ba3f.txt")

{'0': ['3'],
 '1': ['0'],
 '2': ['1', '6'],
 '3': ['2'],
 '4': ['2'],
 '5': ['4'],
 '6': ['5', '8'],
 '7': ['9'],
 '8': ['7'],
 '9': ['6']}

In [24]:
import random

In [25]:
def unexplored_edges(G):
    for elt in (G.values()):
        if len(elt) > 0:
            return True
    return False


In [26]:
def EulerianCycle(G):
    cycle = []
    start = random.choice(list(G.keys()))
    cycle.append(start)
    next = G[start].pop()
    while next != start:
        cycle.append(next)
        next = G[next].pop()
    # falta chequear si hay edges sin usar y construir Cycle'
    while unexplored_edges(G):
        # encontrar newStart
        for elt in cycle:
            if len(G[elt]) > 0:
                newStart = elt
                break
        splice = cycle.index(newStart)
        newcycle = cycle[splice:] + cycle[:splice] + [newStart]
        next = G[newStart].pop()
        while next != newStart:
            newcycle.append(next)
            next = G[next].pop()
            cycle = newcycle
            
    return G, cycle

In [27]:
G = read_ba3f("ba3f.txt")

newG, cycle = EulerianCycle(G)

In [28]:
newG

{'0': [],
 '1': [],
 '2': [],
 '3': [],
 '4': [],
 '5': [],
 '6': [],
 '7': [],
 '8': [],
 '9': []}

In [29]:
list(newG.values())

[[], [], [], [], [], [], [], [], [], []]

In [30]:
"->".join(cycle)

'1->0->3->2->6->8->7->9->6->5->4->2'

Try assembling a circular genome (same as in https://rosalind.info/problems/ba3d/, with the ends stiched together).

```
4
AAGATTCTCTAC
```

```
AAG -> AGA
AGA -> GAT
ATT -> TTC
CTA -> TAC
CTC -> TCT
GAT -> ATT
TCT -> CTA,CTC
TTC -> TCT
```

Then add 

```
TAC -> ACA
ACA -> CAA
CAA -> AAG
```

to simulate a circular genome

In [31]:
G = read_ba3f("eul-test.txt")

newG, cycle = EulerianCycle(G)

In [32]:
"->".join(cycle+[cycle[0]])

'AGA->GAT->ATT->TTC->TCT->CTC->TCT->CTA->TAC->ACA->CAA->AAG->AGA'